In [1]:
import pandas as pd
import time 
from tqdm import tqdm
import warnings
import numpy as np
from statsmodels.tsa.arima_model import ARMA
from itertools import product
from datetime import datetime, timedelta
import calendar
import math
#tqdm(label_list).close()
warnings.filterwarnings('ignore')
#读取数据
all_data = pd.read_csv('data_use.csv')
all_data['时间'] = pd.to_datetime(all_data['时间'])
all_data['价格'] = all_data['价格'].astype('int32')
#获取标签（标签形式：省市/城市/区域）
temp1 = all_data['标签'].values.tolist()
label_dict = {}.fromkeys(temp1).keys()
label_list = list(label_dict)
data_display = pd.DataFrame(columns = ['标签', '时间', '价格'])

for label_name in tqdm(label_list):
#for label_name in label_list[0:10]:
    test_data1 = all_data[all_data.标签 == label_name]
    #设置最小数据量，对于数据量小于最小数据量的区域不作后面3个月的房价预测
    min_data_num = 10
    if len(test_data1) < min_data_num:
        test_data1 = test_data1.sort_values('时间')
        test_data1 = test_data1.reset_index(drop = True)        
        data_display = pd.concat([data_display,test_data1], ignore_index = True)
    else:
        #print(test_data1)
        a1 = test_data1.iloc[0,1].year
        a2 = test_data1.iloc[0,1].month
        #判断爬取的房价数据在时间维度上是否连续，若不连续，则向后追溯到连续，将开始连续的日期作为预测数据的开始
        for i in range(len(test_data1)):
            num_month = (len(test_data1)-i-1)%12
            num_year = math.floor((len(test_data1)-i-1)/12)
            a3 = test_data1.iloc[len(test_data1)-1-i,1].month + num_month
            a4 = test_data1.iloc[len(test_data1)-1-i,1].year + num_year
            if a3 > 12:
                temp1 = a3%12
                temp2 = math.floor(a3/12)
                a3 = temp1
                a4 = a4 + temp2
            if a3 == a2:
                if a4 == a1:
                    test_data1 = test_data1.iloc[:len(test_data1)-i,:]
                    break
        price_nunique = test_data1['价格'].value_counts()
        test_data2 = test_data1[['时间','价格']]
        test_data2 = test_data2.sort_values('时间')
        test_data2 = test_data2.reset_index(drop = True)
        test_data2.index = test_data2['时间']
        test_data2 = test_data2.drop('时间',axis =1)
        test_data2['价格'] = test_data2['价格'].astype('float32')
        # 设置参数范围
        ps = range(0, 3)
        qs = range(0, 3)
        parameters = product(ps, qs)
        parameters_list = list(parameters)
        # 寻找最优ARMA模型参数，即best_aic最小
        results = []
        best_aic = float("inf") # 正无穷
        for param in parameters_list:
            #print(param)
            try:
                model = ARMA(test_data2['价格'],order=(param[0], param[1])).fit()
            except :
                #print('参数错误:', param)
                continue
            aic = model.aic
            if aic < best_aic:
                best_model = model
                best_aic = aic
                best_param = param
            results.append([param, model.aic])
        # 设置future_month，需要预测的时间date_list
        df_month2 = test_data2[['价格']]
        future_month = 3
        last_month = pd.to_datetime(df_month2.index[len(df_month2)-1])
        #print(last_month)
        date_list = []
        for i in range(future_month):
            # 计算本月有多少天
            year = last_month.year
            month = last_month.month
            next_month_days = calendar.monthrange(year, month)[1]
            if month == 12:
                month = 1
                year = year+1
            else:
                month = month + 1
            #print(next_month_days)
            last_month = last_month + timedelta(days=next_month_days)
            date_list.append(last_month)
        # 添加未来要预测的3个月
        future = pd.DataFrame(index=date_list, columns= df_month2.columns)
        df_month2 = pd.concat([df_month2, future])
        #判断价格是否是唯一值，若为唯一，后续3个月则继续保持该唯一值
        if len(price_nunique) == 1:
            df_month2 = df_month2.reset_index()
            df_month2['价格'] = df_month2['价格'].fillna(value = df_month2.loc[0,'价格'])
            df_month2['标签'] = label_name
            df_month2 = df_month2.rename(columns = {'index':'时间'})
            df_month2['价格'] = df_month2['价格'].astype('int32')
            data_one = df_month2[['标签','时间','价格']]
            data_display = pd.concat([data_display,data_one], ignore_index = True)
        else:
            df_month2['forecast'] = best_model.predict(start=0, end=len(df_month2))
            # 第一个元素不正确，设置为NaN
            df_month2['forecast'][0] = np.NaN
            #print(df_month2)
            df_month2 = df_month2.reset_index()
            a = len(df_month2)-3
            b = len(df_month2)-2
            c = len(df_month2)-1
            values = {a:df_month2.loc[a,'forecast'],
                     b:df_month2.loc[b,'forecast'],
                     c:df_month2.loc[c,'forecast'],}  
            df_month2['价格'] = df_month2['价格'].fillna(value = values)
            df_month2 = df_month2.drop('forecast',axis =1)
            df_month2['标签'] = label_name
            df_month2 = df_month2.rename(columns = {'index':'时间'})
            df_month2['价格'] = df_month2['价格'].astype('int32')
            data_one = df_month2[['标签','时间','价格']]
            data_display = pd.concat([data_display,data_one], ignore_index = True)
        
print(data_display)
#结果输出
data_display.to_csv('data_display.csv',index = 0)

100%|██████████████████████████████████████████████████████████████████████████████| 2499/2499 [22:10<00:00,  3.05it/s]


                标签         时间     价格
0        黑龙江/大庆/肇州 2019-08-01   3334
1        黑龙江/大庆/肇州 2019-09-01   3370
2        黑龙江/大庆/肇州 2019-10-01   3360
3        黑龙江/大庆/肇州 2019-11-01   3350
4        黑龙江/大庆/肇州 2019-12-01   3419
5        黑龙江/大庆/肇州 2020-01-01   3517
6        黑龙江/大庆/肇州 2020-02-01   3515
7        黑龙江/大庆/肇州 2020-03-01   3546
8        黑龙江/大庆/肇州 2020-04-01   3477
9       黑龙江/大庆/萨尔图 2015-04-01   5330
10      黑龙江/大庆/萨尔图 2015-05-01   5414
11      黑龙江/大庆/萨尔图 2015-06-01   5453
12      黑龙江/大庆/萨尔图 2015-07-01   5545
13      黑龙江/大庆/萨尔图 2015-08-01   5560
14      黑龙江/大庆/萨尔图 2015-09-01   5510
15      黑龙江/大庆/萨尔图 2015-10-01   5555
16      黑龙江/大庆/萨尔图 2015-11-01   5552
17      黑龙江/大庆/萨尔图 2015-12-01   5550
18      黑龙江/大庆/萨尔图 2016-01-01   5560
19      黑龙江/大庆/萨尔图 2016-02-01   5553
20      黑龙江/大庆/萨尔图 2016-03-01   5559
21      黑龙江/大庆/萨尔图 2016-04-01   5591
22      黑龙江/大庆/萨尔图 2016-05-01   5548
23      黑龙江/大庆/萨尔图 2016-06-01   5571
24      黑龙江/大庆/萨尔图 2016-07-01   5597
25      黑龙江/大庆/萨尔图 2016-08-01   5694
2